# アンサンブル学習

- ブレンディング
- バギング
- スタッキング

In [1]:
# 必要なモジュールをインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings #ワーニング関連のモジュール？
warnings.filterwarnings('ignore') #ワーニングが消える？
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_diabetes
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import scipy.stats
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [2]:
df = pd.read_csv("train_1.csv")

# 目的変数
y = df.loc[:,["SalePrice"]]

#説明変数
X   = df.loc[:,["GrLivArea","YearBuilt"]]


In [3]:
lr = LinearRegression()
svm = SVR()
dtr = DecisionTreeRegressor()
rfr = RandomForestRegressor()

In [4]:
# トレーニングデータ分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## 【問題1】ブレンディングのスクラッチ実装

### ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証データに対する平均二乗誤差（MSE）が小さくなることを指します。

## ブレンディングとは

ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。

- 手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）
- ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）
- 入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。
回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。

分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。

In [5]:
"""ブレンディング"""
class scratch_brend():

    def train(self,X_train,y_train,X_test,y_test):
        print("------------")
        print("デフォルトのMSE")
        print("------------")
        models = [lr,svm,dtr,rfr]
        model_names =["Logistic Regression","SVM","Decision Tree","Random Forest"]
        ans_mean = 0
        count = 0
        for model,model_name in zip(models,model_names):
            print(model_name)
            model.fit(X_train,y_train)
            y_pred = model.predict(X_test)
            model_mean = mean_squared_error(y_test,y_pred)
            print(model_mean)
            print("------------")
            ans_mean += model_mean
            count += 1
        ans = ans_mean/count
        print("ブレンディング後のMSE")
        return ans

    def trans(self,X_train,y_train,X_test,y_test):
        print("------------")
        print("標準化したMSE")
        print("------------")
        X_train_trans = scipy.stats.zscore(X_train)
        X_test_trans = scipy.stats.zscore(X_test)
        models = [lr,svm,dtr,rfr]
        model_names =["Logistic Regression","SVM","Decision Tree","Random Forest"]
        ans_mean = 0
        ans_mean = 0
        count = 0
        for model,model_name in zip(models,model_names):
            print(model_name)
            model.fit(X_train_trans,y_train)
            y_pred = model.predict(X_test_trans)
            model_mean = mean_squared_error(y_test,y_pred)
            print(model_mean)
            print("------------")
            ans_mean += model_mean
            count += 1
        ans = ans_mean/count
        print("ブレンディング後のMSE")
        return ans

    def log(self,X_train,y_train,X_test,y_test):
        print("------------")
        print("対数変換したMSE")
        print("------------")
        X_train_log = X_train.apply(np.log)
        X_test_log = X_test.apply(np.log)
        models = [lr,svm,dtr,rfr]
        model_names =["Logistic Regression","SVM","Decision Tree","Random Forest"]
        ans_mean = 0
        ans_mean = 0
        count = 0
        for model,model_name in zip(models,model_names):
            print(model_name)
            model.fit(X_train_log,y_train)
            y_pred = model.predict(X_test_log)
            model_mean = mean_squared_error(y_test,y_pred)
            print(model_mean)
            print("------------")
            ans_mean += model_mean
            count += 1
        ans = ans_mean/count
        print("ブレンディング後のMSE")
        return ans

scb = scratch_brend()

In [6]:
print(scb.train(X_train,y_train,X_test,y_test))

------------
デフォルトのMSE
------------
Logistic Regression
1580858237.456167
------------
SVM
5175145038.668915
------------
Decision Tree
2042664715.5924656
------------
Random Forest
1451597763.1440623
------------
ブレンディング後のMSE
2562566438.7154026


In [7]:
print(scb.trans(X_train,y_train,X_test,y_test))

------------
標準化したMSE
------------
Logistic Regression
1581582628.6818948
------------
SVM
5176763974.21277
------------
Decision Tree
3375402398.8912673
------------
Random Forest
1683411954.0409417
------------
ブレンディング後のMSE
2954290238.9567184


In [8]:
print(scb.log(X_train,y_train,X_test,y_test))

------------
対数変換したMSE
------------
Logistic Regression
1871291769.8489769
------------
SVM
5174748361.732942
------------
Decision Tree
2100443735.6541095
------------
Random Forest
1419750500.0100358
------------
ブレンディング後のMSE
2641558591.8115163


## 【問題2】バギングのスクラッチ実装

### バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

## バギングとは

バギングは入力データの選び方を多様化する方法です。訓練データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。

推定結果の平均をとる部分はブレンディングと同様の実装になります。

In [9]:
"""バギング"""

class scratch_bagi():
    def bagi(self,X,y,X_test,y_test):
        print("------------")
        print("バギング")
        #print("------------")
        # Xのインデックス分の配列を作る（インデックス番号を指定するため）
        X_np = np.arange(X.shape[0])
        # 最終的なMSEを保管するための変数
        fainal_ans =0
        fainal_count = 0
        X = np.array(X)
        y = np.array(y)
        # サンプルの配列の数分だけ繰り返す
        for i in range(5):
            # 重複ありのtrainデータを作る
            X_train=np.array([])
            y_train=np.array([])
            select = np.random.choice(X_np, size=int(X.shape[0]*0.8))
            for j in range(len(select)):
                X_train = np.append(X_train,X[int(select[j])])
                y_train = np.append(y_train,y[int(select[j])])
            X_train = X_train.reshape(-1,2)

            models = [lr,svm,dtr,rfr]
            model_names =["Logistic Regression","SVM","Decision Tree","Random Forest"]
            ans_mean = 0
            count = 0
            for model,model_name in zip(models,model_names):
                print("------------")
                print(model_name)
                model.fit(X_train,y_train)
                y_pred = model.predict(X_test)
                model_mean = mean_squared_error(y_test,y_pred)
                print(model_mean)
                #print("------------")
                ans_mean += model_mean
                count += 1
            ans = ans_mean/count
            fainal_ans += ans
            fainal_count += 1
            print("------------")
            print("MSE")
            print(ans)
        last_ans = fainal_ans/fainal_count
        print("------------")
        print("last_ans")
        return last_ans

scb = scratch_bagi()



In [10]:
print(scb.bagi(X,y,X_test,y_test))

------------
バギング
------------
Logistic Regression
1595905365.7431376
------------
SVM
5122759112.734876
------------
Decision Tree
1395278361.6503043
------------
Random Forest
1231856490.9752548
------------
MSE
2336449832.7758927
------------
Logistic Regression
1632311437.2265377
------------
SVM
5183005407.582917
------------
Decision Tree
1168033910.533105
------------
Random Forest
993871730.2329385
------------
MSE
2244305621.3938746
------------
Logistic Regression
1695416427.6895301
------------
SVM
5152920017.852147
------------
Decision Tree
1034977415.6712328
------------
Random Forest
750554424.9582678
------------
MSE
2158467071.5427947
------------
Logistic Regression
1574124263.847179
------------
SVM
5045012747.685332
------------
Decision Tree
964942289.6230024
------------
Random Forest
1038411439.6805913
------------
MSE
2155622685.209026
------------
Logistic Regression
1676049543.2117896
------------
SVM
5174913269.347317
------------
Decision Tree
1378395093.048

## 【問題3】スタッキングのスクラッチ実装

### スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください

## スタッキングとは

### スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは $K_0=3, M_0=2$ 程度にします。

《学習時》

（ステージ $0$ ）

- 訓練データを $K_0$ 個に分割する。
- 分割した内の $(K_0 - 1)$ 個をまとめて訓練データ、残り $1$ 個を推定用データとする組み合わせが $K_0$ 個作れる。
- あるモデルのインスタンスを $K_0$ 個用意し、異なる訓練データを使い学習する。
- それぞれの学習済みモデルに対して、使っていない残り $1$ 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）
- さらに、異なるモデルのインスタンスも $K_0$ 個用意し、同様のことを行う。モデルが $M_0$ 個あれば、 $M_0$ 個のブレンドデータが得られる。

（ステージ $n$ ）

- ステージ $n-1$ のブレンドデータを$M_{n-1}$ 次元の特徴量を持つ訓練データと考え、 $K_n$ 個に分割する。以下同様である。

（ステージ $N$ ）＊最後のステージ

- ステージ $N-1$ の $M_{N-1}$ 個のブレンドデータを$M_{N-1}$ 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

《推定時》

（ステージ $0$ ）

- テストデータを $K_0×M_0$ 個の学習済みモデルに入力し、$K_0×M_0$ 個の推定値を得る。これを $K_0$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $n$ ）

- ステージ $n-1$ で得たブレンドテストを $K_n×M_n$ 個の学習済みモデルに入力し、$K_n×M_n$ 個の推定値を得る。これを $K_n$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $N$ ）＊最後のステージ

ステージ $N-1$ で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

In [11]:
X = np.array(X)
y = np.array(y)
# トレーニングデータ分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
"""スタッキング"""

class Staking():

    def __init__(self,max_depth,splits,models):

        self.max_depth = max_depth
        self.n_splits = splits
        self.models = models
        self.fit_models = []

    def blending(self,X,y,model):
        # blendの初期化
        blend = np.zeros(len(X))
        # データの分割
        kf = KFold(n_splits=self.n_splits,shuffle=False)
        # 分割されたインデックスでループ
        for train_id,test_id in kf.split(X):
            # データ分割
            X_train = X[train_id]
            X_test  = X[test_id]
            y_train = y[train_id]
            y_test  = y[test_id]
            y_train = y_train.ravel()
            y_test  = y_test.ravel()

            # 学習
            model.fit(X_train,y_train)

            #学習ずみのモデルを保存
            self.fit_models.append(model)

            # テストデータのインデックスに予測値を格納
            #print(model.predict(X_test))
            blend[test_id] = model.predict(X_test)

        return blend

    def fit(self,X,y,depth):

        # 現在の深さ
        self.depth = depth
        # 最大の深さまで到達していれば、その深さを学習させて処理を終了
        if self.depth == self.max_depth:
            # 当該深でのモデルを取得
            self.model = self.models[self.depth]
            # 学習
            self.model.fit(X,y)
            return

        # 当該深さでのモデルを取得
        models = self.models[self.depth]
        self.blend = np.zeros([len(X),len(models)])

        # この階層の全てのモデルで学習
        for i,model in enumerate(models):
            _blend = self.blending(X,y,model)
            blend = _blend

        # 再帰学習
        # コンストラクタ生成の際の引数は同じ
        self.stk = Staking(self.max_depth,self.n_splits,self.models)

        # 学習実行の際は、ブレンドデータを説明変数として渡す
        self.stk.fit(self.blend,y,depth+1)

    def predict(self,X):
        # 最大の深さの場合は最終的な予測値を出力するのみ
        if self.depth == self.max_depth:
            pred = self.model.predict(X)
            return pred

        else:
            # 予測値を０で初期化
            self.pred = np.zeros(len(X))
            # 次の階層に渡すブレンドデータ
            self.blend = np.zeros([len(X),len(self.models[self.depth])])
            # この階層の学習ずみのモデルでループ
            count = 0
            for model in self.fit_models:
                # 予測し、０で初期化している予測値に加算
                self.pred += model.predict(X)
                # 1種類のモデルの学習の終了
                count += 1
                if count%self.n_splits == 0:
                    # 予測値を加算してきたので割って平均値を算出
                    self.pred = self.pred/self.n_splits
                    # その平均値を次の階層で使用する説明変数に格納
                    self.blend[:,int(count/self.n_splits)-1] = self.pred
                    # 次の種類のモデルでの予測のため、予測値は初期化しておく
                    self.pred = np.zeros(len(X))
            # 次の階層の予測関数
            pred = self.stk.predict(self.blend)
            return pred

In [13]:
models = {
    0:[LinearRegression(),DecisionTreeRegressor(),RandomForestRegressor()],
    1:LinearRegression()
}

stk = Staking(max_depth=1,splits=5,models=models)
stk.fit(X_train,y_train,0)

In [14]:
prediction = stk.predict(X_test)

In [15]:
# Rating
mse = mean_squared_error(y_test,prediction)
print('MSE : {:.3f}'.format(mse))

MSE : 6599382160.934
